In [1]:
import pandas as pd
import numpy as np

In [2]:
def estimateparamsbayes(X,questions,allbest,allworst,w0=[],weights=[],tol=.0001,maxiter=100):
    tol=.001
    val=1
    [numofq,nitems]=X.shape
    if len(w0)<nitems or (np.abs(w0)>4).any():
        w=np.zeros(nitems)    
    else:
        w=w0.copy()
    if len(weights)<numofq:
        weights=np.ones(numofq)
    iternum=0
    nb=np.bincount(allbest,weights,minlength=nitems)
    nw=np.bincount(allworst,weights,minlength=nitems)
    try:
        while val>tol and iternum<maxiter:
            zb=np.sum(np.exp(w[questions]),1)
            zw=np.sum(np.exp(-w[questions]),1)
            Pb=((1/zb*X.T).T*np.exp(w))#broadcasting
            Pw=((1/zw*X.T).T*np.exp(-w))#broadcasting
            wPb=(Pb.T*weights).T
            wPw=(Pw.T*weights).T
            gradb=nb-np.sum(wPb,0)
            gradw=-nw+np.sum(wPw,0)
            grad=-(gradb+gradw)
            Hb=np.dot(wPb.T,Pb)
            Hbd=-np.sum(wPb*(1-Pb),0)
            Hb+=-np.diag(np.diag(Hb))+np.diag(Hbd)
            Hw=np.dot(wPw.T,Pw)
            Hwd=-np.sum(wPw*(1-Pw),0)
            Hw+=-np.diag(np.diag(Hw))+np.diag(Hwd)
            H=-(Hb+Hw)
            gradl=grad[:-1]
            Hl=H[:-1,:-1]
            Hinvgradl=np.linalg.solve(Hl,gradl)
            w[:-1]=w[:-1]-Hinvgradl
            val=np.dot(gradl,Hinvgradl)
            iternum+=1
    except np.linalg.linalg.LinAlgError:
        return np.random.randn(nitems)/4
    if np.isnan(w).any():
        raise Exception('w contains nan')
    return w

In [3]:
def estimateparams(X,questions,nb,nw,w0=[],tol=.0001,maxiter=100,hessian=0):
    tol=.001
    val=1
    [numofq,nitems]=X.shape
    if len(w0)<nitems or (np.abs(w0)>4).any():
        w=np.zeros(nitems)    
    else:
        w=w0.copy()
    iternum=0
    inplay=np.where(np.bincount(questions.flatten(),minlength=nitems)>0)[0]
    notinplay=np.where(np.bincount(questions.flatten(),minlength=nitems)==0)[0]
    try:
        while val>tol and iternum<maxiter:
            zb=np.sum(np.exp(w[questions]),1)
            zw=np.sum(np.exp(-w[questions]),1)
            Pb=((1/zb*X.T).T*np.exp(w))#broadcasting
            Pw=((1/zw*X.T).T*np.exp(-w))#broadcasting
            gradb=nb-np.sum(Pb,0)
            gradw=-nw+np.sum(Pw,0)
            grad=-(gradb+gradw)
            Hb=np.dot(Pb.T,Pb)
            Hbd=-np.sum(Pb*(1-Pb),0)
            Hb+=-np.diag(np.diag(Hb))+np.diag(Hbd)
            Hw=np.dot(Pw.T,Pw)
            Hwd=-np.sum(Pw*(1-Pw),0)
            Hw+=-np.diag(np.diag(Hw))+np.diag(Hwd)
            H=-(Hb+Hw)
            gradl=grad[inplay[:-1]]
            Hl=H[inplay[:-1],:][:,inplay[:-1]]
            #Hlinv=np.linalg.pinv(Hl)
            #Hinvgradl=np.dot(Hlinv,gradl)
            Hinvgradl=np.linalg.solve(Hl,gradl)
            w[inplay[:-1]]=w[inplay[:-1]]-Hinvgradl
            val=np.dot(gradl,Hinvgradl)
            iternum+=1
    except np.linalg.linalg.LinAlgError:
        print('there')
        if hessian:
            
            return np.random.randn(nitems)/4,np.eye(nitems-1)
        else:
            return np.random.randn(nitems)/4
    if np.isnan(w).any():
        print('here')
        return np.random.randn(nitems)/4
        #raise Exception('w contains nan')
    w[notinplay]=w[np.random.choice(inplay,notinplay.shape[0])]+.001
    
    if hessian:
        H=np.eye(nitems-1)
        H[inplay[:-1],:][:,inplay[:-1]]=Hl
        return w,H
    else:
        return w

In [4]:
k=3
filename="uncert-120v20k3-mis"
data=pd.read_csv('Results/choice_data-'+filename+'.csv')
bhist=pd.read_csv('Results/bhist-'+filename+'.csv')
itemcol=[]
itemnum=120
for i in range(1,itemnum+1):
    itemcol+=['Item_'+str(i)]
iters=int(np.max(bhist['Iter']))
respnums=(bhist[bhist['Iter']==1]['RespNum'].get_values()).astype(int)
resplen=len(respnums)
choicecol=['Choice1','Choice2', 'Choice3', 'Choice4', 'Choice5']

In [398]:
iternum=65
test=data[data['Iter']==iternum]

In [399]:
questions=test[choicecol].get_values()-1
best=test['Best'].get_values()-1
worst=test['Worst'].get_values()-1

In [400]:
np.bincount(questions[:rows].flatten())[[72,62,48,105,66,108]]

array([555, 396, 141, 759, 111,  90])

In [401]:
[numofq,q]=questions.shape
X=np.zeros((numofq,itemnum))
for z,row in enumerate(questions):
    X[z,row]=1
rows=12*300
begin=0
nb=np.bincount(best[begin:rows],minlength=itemnum)
nw=np.bincount(worst[begin:rows],minlength=itemnum)
param=estimateparams(X[begin:rows],questions[begin:rows],nb,nw)

In [402]:
np.argsort(param)[::-1][:10]

array([105,  72,  62, 112, 104,  75,  53,  36,  54,  71])

In [5]:
data1=pd.read_csv('HB_120items.csv')
utilsall=data1.get_values()[:,2:]
realmean=np.mean(utilsall,0)
topk=np.argsort(realmean)[::-1][:k]

In [6]:
topk

array([72, 62, 48])

In [7]:
np.argsort(realmean)[::-1]

array([ 72,  62,  48, 105, 104, 112,  71,  54,  36,  53,  75, 114,  59,
        76,  70,  16,  42,  17,  20,  57, 113,  14,  50,  23,  80, 110,
       103,  43,   5,   9,  22,  10,  19,   4,  91,  77,  88,  82,  56,
        58,  32,  83,  31,   3,  73,  95,  45, 119,  93, 106, 109,  29,
        60,  84,  15,   8,   6,  63,  66,  89,  40,  35, 111,  41, 115,
        90,  28,  30,  39,  21,  27,  74,  26,   0,  38,  49,  44,  11,
        79,  37, 107, 118,  69,  87, 100,  18,  61,  98,  25, 108,  96,
        33,  52,  13,  78,  67,  64,  94,  55,  47,  99,   2,  46,  68,
        81, 117, 101,   7,  12,  65,  51, 102, 116,  85,  97,  92,   1,
        86,  34,  24])

In [343]:
testutil=utilsall[4]

In [345]:
np.argsort(testutil)[::-1]

array([ 82,   9,  76,  36,  39,  54,  60,  21, 113,  73,  23,  57,  66,
        77,  75, 110,  88,  70,  55,  59,  84,  99, 105,  89,  53,  20,
        71,  16,  14,   8, 103, 104,  32,  29,  83,  56,  43,   6,   5,
        30, 115,  49,  61,  17,  40,  87,  42,  35,  50,   4,  69, 109,
         3,  90, 106,  31,  25,  19, 119,  79,  74,  10,  63,  27,  44,
       112,  78,  52,  95,  93,  41,  46,  28,  26,  80, 107, 114,   0,
        64,  22, 118,  38, 100,  12,  86,  67,  37,  72,  48,  62,  58,
       108,  18,  92,  94,  91,  98, 111,  15,  13, 116,  33,  97,  96,
        85,   2,  11,   1, 117,   7,  45,  81,  47,  51,  68, 102, 101,
        34,  65,  24])